In [3]:
import os
import pickle
import numpy as np
import gym
import load_policy_pytorch
import models
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


In [4]:
def train_validate(model, dataloader, optim, loss_fn, epoch,train):
    loss_all = 0
    model.train() if train else model.eval()
    for batch_idx, (x, y) in enumerate(dataloader):

        pred_act = model(x)
        loss = loss_fn(pred_act, y)

        if train:
            optim.zero_grad()
            loss.backward()
            optim.step()
       # if batch_idx % 50 == 0:
        #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
         #       epoch, batch_idx * len(x), len(dataloader.dataset),
         #              100. * batch_idx / len(dataloader), loss.item() / len(x)))



        loss_all += loss.item()
    return loss_all / len(dataloader.dataset)


In [9]:
expert_policy_file = './experts/Humanoid-v2.pkl '
envname = 'Humanoid-v2'
render = False
max_timesteps = 100
num_rollouts = 20
batch_size = 50



In [12]:
expert_data = pickle.load(open(os.path.join('expert_data/' + envname + '.pkl'), 'rb'))
expert_obs = expert_data['observations']
expert_act = expert_data['actions']

print(expert_obs.shape)
print(expert_act.shape)

expert_obs_train, expert_obs_test, expert_act_train, expert_act_test = train_test_split(expert_obs, expert_act, test_size = 0.2)


expert_obs_train = torch.from_numpy(expert_obs_train).type(torch.FloatTensor)
expert_act_train = torch.from_numpy(expert_act_train).squeeze(1).type(torch.FloatTensor)
expert_obs_test = torch.from_numpy(expert_obs_test).type(torch.FloatTensor)
expert_act_test = torch.from_numpy(expert_act_test).squeeze(1).type(torch.FloatTensor)

dataset_train = TensorDataset(expert_obs_train, expert_act_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = TensorDataset(expert_obs_test, expert_act_test)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

(4000, 376)
(4000, 1, 17)


In [13]:
bc_model = models.bc(expert_obs.shape[1], expert_act.shape[1])

optim = Adam(bc_model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
n_epochs = 50


train_loss = []
val_loss = []
for epoch in range(0, n_epochs):
    t_loss = train_validate(bc_model, dataloader_train, optim, loss_fn, epoch, train=True)
    train_loss.append(t_loss)
    v_loss = train_validate(bc_model, dataloader_val, optim, loss_fn, epoch, train=False)
    val_loss.append(v_loss)


/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([50, 17])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
sns.set()
plt.plot(np.arange(len(train_loss)), train_loss, label='train')
plt.plot(np.arange(len(val_loss)), val_loss, label='val')
plt.title('Training')
plt.xlabel('Step')
plt.ylabel('Negative log loss (bits)')
plt.legend()
plt.grid(True)
